In [38]:
import sys
sys.path.append('../src')

import network as nw
import road as rd
import junction as jn
import traffic_lights as tl
import torch
import openGLUtils as glutils
import openGLUtilsGif as glgif
import matplotlib.pyplot as plt
import FV_schemes as fv
import scipy.integrate as integrate
import numpy as np
import roundabout as rb
import torch
import generate_kvadraturen as gk
import bus
import loading_json as load

In [39]:
T = 100
# network = gk.generate_kvadraturen_minimal_junctions(T)

network = gk.generate_kvadraturen_w_roundabout(T)

In [5]:
# import json
# print("Loading results...")
# f = open("kvadraturen_roundabout.json")
# data = json.load(f)
# f.close()
# densities = data[0]
# queues = data[1]
# bus_lengths = data[2]
# bus_delays = data[3]

Loading results...


In [40]:
ids_bw = ["lundsbro_bw", "elvegata_bw", "tollbod_2bw", "tollbod_1bw", "v_strand_5bw", 
          "v_strand_4bw", "v_strand_3bw", "v_strand_2bw", "v_strand_1bw"]            
            

stops_bw = [("tollbod_2bw", 50), ("tollbod_1bw", 90), ("tollbod_1bw", 230), ("v_strand_1bw", 25)]

times_bw = [40, 130, 190, 250]

bus_bw = bus.Bus(ids_bw, stops_bw, times_bw, network, id = "2", start_time = 0.0)

ids_fw = ["v_strand_1fw", "h_w_2", "festning_3fw", "festning_4fw", "tollbod_2fw",
          "elvegata_fw", "lundsbro_fw"]

stops_fw = [("h_w_2", 130), ("festning_4fw", 40), ("tollbod_2fw", 25), 
            ("tollbod_2fw", 260)]
times_fw = [30, 110, 130, 230]


bus_fw = bus.Bus(ids_fw, stops_fw, times_fw, network, id = "1")


times_bw_2 = [240, 330, 390, 450]

times_fw_2 = [530, 610, 630, 830]

bus_bw_2 = bus.Bus(ids_bw, stops_bw, times_bw, network, id = "3", start_time = 200.0)
bus_fw_2 = bus.Bus(ids_fw, stops_fw, times_fw, network, id = "4", start_time = 500.0)

In [41]:
# Create actual network
roads = network.roads
junctions = network.junctions
T = network.T
roundabouts = network.roundabouts
# Don't store the densities 
T = 100
# bus_network = nw.RoadNetwork(roads, junctions, T, [bus_fw, bus_bw], store_densities = False)
bus_network = nw.RoadNetwork(roads, junctions, T, busses=[bus_fw, bus_bw, bus_fw_2, bus_bw_2], roundabouts=roundabouts,
                             store_densities = True)

In [42]:
densities, _, _, _  = bus_network.solve_cons_law()

Bus 2 reached bus stop 0 at time 19.600006103515625, should wait for 29.75235939025879 seconds
Bus 1 reached bus stop 0 at time 27.40001106262207, should wait for 29.48542022705078 seconds
Bus 1 reached bus stop 1 at time 86.7474365234375, should wait for 29.866254806518555 seconds
Bus 2 reached bus stop 1 at time 92.08149719238281, should wait for 37.43041229248047 seconds


In [47]:
for i, road in enumerate(bus_network.roads):
    if road.id == "vs_mainline_4":
        print(i)

42


In [58]:
jnc_idx = 0
index = 39 + jnc_idx
times = densities[index].keys()
for i, t in enumerate(times):
    if i > 10:
        break
    print(f"t = {t}, densities = {densities[index][t]}")

# 42 seem to stabilize around 0.1798
# 41 seem to stabilize around 0.9747, 0.9747, 0.6386, 0.5232, 0.5046, 0.5046
# 40 seem to stabilize around 0.4921, 0.4921, 0.4895, 0.4860, 0.4821, 0.4821
# 39 seem to stabilize around 0.1339

t = 0, densities = tensor([0., 0., 0., 0., 0., 0.])
t = 0.5999999642372131, densities = tensor([0.0570, 0.0570, 0.0000, 0.0000, 0.0000, 0.0000], grad_fn=<CopySlices>)
t = 1.1999999284744263, densities = tensor([0.0598, 0.0598, 0.0280, 0.0265, 0.0262, 0.0262], grad_fn=<CopySlices>)
t = 1.7999999523162842, densities = tensor([0.0691, 0.0691, 0.0440, 0.0412, 0.0403, 0.0403], grad_fn=<CopySlices>)
t = 2.3999998569488525, densities = tensor([0.0748, 0.0748, 0.0566, 0.0536, 0.0524, 0.0524], grad_fn=<CopySlices>)
t = 2.999999761581421, densities = tensor([0.0818, 0.0818, 0.0658, 0.0631, 0.0619, 0.0619], grad_fn=<CopySlices>)
t = 3.5999996662139893, densities = tensor([0.0886, 0.0886, 0.0739, 0.0712, 0.0700, 0.0700], grad_fn=<CopySlices>)
t = 4.199999809265137, densities = tensor([0.0947, 0.0947, 0.0813, 0.0786, 0.0773, 0.0773], grad_fn=<CopySlices>)
t = 4.799999713897705, densities = tensor([0.1000, 0.1000, 0.0881, 0.0853, 0.0841, 0.0841], grad_fn=<CopySlices>)
t = 5.0, densities = tensor([0.

In [31]:
# Investigate the roundabout
for roundabout in bus_network.roundabouts:
    # for j in roundabout.junctions:
    #     print(j.)
    for mainline in roundabout.mainline_roads:
        print(mainline.id)
    for secondary in roundabout.secondary_inroads:
        try:
            print(secondary.id)
        except:
            print("None")
            print(secondary.inflow_fnc(0.2))

    for secondary in roundabout.secondary_outroads:
        try:
            print(secondary.id)
        except:
            print("None")

    print("\n--------------------------------\n")

vs_mainline_1
vs_mainline_2
vs_mainline_3
vs_mainline_4
v_strand_1fw
None
tensor(0.1042)
None
tensor(0.0264)
None
tensor(0.0317)
v_strand_1bw
None
None
None

--------------------------------

fn_mainline_1
fn_mainline_2
fn_mainline_3
festning_1fw
None
tensor(0.0708)
None
tensor(0.0708)
festning_1bw
None
None

--------------------------------



In [37]:
for j in bus_network.roundabouts[0].junctions:
    print(j.mainline_in.id)
    print(j.mainline_out.id)
    print(j.mainline_in.left_pos, j.mainline_in.right_pos)
    try:
        print(j.secondary_in.id)
        print(j.secondary_out.id)
    except:
        print("External secondary roads:")
        print(f"Influx: {j.secondary_in.inflow_fnc(0.2)}")


vs_mainline_1
vs_mainline_2
(0, 0) (0.15, -0.3)
v_strand_1fw
v_strand_1bw
vs_mainline_2
vs_mainline_3
(0.15, -0.3) (0, -0.6)
External secondary roads:
Influx: 0.1041666716337204
vs_mainline_3
vs_mainline_4
(0, -0.6) (-0.15, -0.3)
External secondary roads:
Influx: 0.02638889104127884
vs_mainline_4
vs_mainline_1
(-0.15, -0.3) (0, 0)
External secondary roads:
Influx: 0.03166666626930237


In [10]:
# for i, road in enumerate(bus_network.roads):
#     print(i, road.id)

# 39, 40, 41, 42 are mainline roads of the first roundabout

In [48]:
index = '39'
times = densities[index].keys()
for i, t in enumerate(times):
    print(f"Time: {t}, In-boundary density: {densities[index][t][0]}")
    print(f"Time: {t}, Out-boundary density: {densities[index][t][-1]}")
    if i > 20:
        break

KeyError: '39'

#### Testing for a minimal roundabout

In [2]:
L = 25 # Length of road
N = 2 # Number of nodes to be used for every 50 meters

# Vestre Strandgate roundabout:
# Four arm roundabout with 4 junctions, where 2 roads are a part of simulation
# Secondary roads can be described by a small object with a queue and an inflow condition
# Creating the mainline
main_speed_limit = torch.tensor([50.0], requires_grad=True)
vs_main_1 = rd.Road(1, L, N, main_speed_limit, [], left_pos=(0,0), right_pos=(0.5, -0.5),
                    inflow = 0.0, id="vs_mainline_1")
vs_main_2 = rd.Road(1, L, N, main_speed_limit, [], left_pos=(0.5, -0.5), right_pos=(0, -1),
                    inflow = 0.0, id="vs_mainline_2")
vs_main_3 = rd.Road(1, L, N, main_speed_limit, [], left_pos=(0, -1), right_pos=(-0.5, -0.5),
                    inflow = 0.0, id="vs_mainline_3")
vs_main_4 = rd.Road(1, L, N, main_speed_limit, [], left_pos=(-0.5, -0.5), right_pos=(0, 0),
                    inflow = 0.0, id="vs_mainline_4")

In [3]:
secondary_gamma = torch.tensor(50 / (25 * 3.6))
max_inflow = fv.flux(torch.tensor(0.5), secondary_gamma)
# Assume constant inflow from all roads
rho_1, rho_2, rho_3 = torch.tensor(0.4),torch.tensor(0.05), torch.tensor(0.05)
inflow_1 = lambda t : fv.flux(rho_1, secondary_gamma)
inflow_2 = lambda t : fv.flux(rho_2, secondary_gamma)
inflow_3 = lambda t : fv.flux(rho_3, secondary_gamma)

In [4]:
max_inflow

tensor(0.1389)

In [5]:
# Secondary roads
vs_fw = rd.Road(2, L, N, main_speed_limit, [], left_pos=(0.0,0.0), right_pos=(0.0,1),
                    inflow = 0.0, id="vs_fw")
vs_bw = rd.Road(2, L, N, main_speed_limit, [], left_pos=(0.0,1), right_pos=(0,0),
                    inflow = 0.0, id="vs_bw")
vs_secondary_1 = rb.RoundaboutRoad(inflow_1, max_inflow)
vs_secondary_2 = rb.RoundaboutRoad(inflow_2, max_inflow)
vs_secondary_3 = rb.RoundaboutRoad(inflow_3, max_inflow)
vs_jnc_1 = rb.RoundaboutJunction(vs_main_1, vs_main_2, 0.7, vs_fw, vs_bw, queue_junction = False)
vs_jnc_2 = rb.RoundaboutJunction(vs_main_2, vs_main_3, 0.7, vs_secondary_1, None, queue_junction = True)
vs_jnc_3 = rb.RoundaboutJunction(vs_main_3, vs_main_4, 0.7, vs_secondary_2, None, queue_junction = True)
vs_jnc_4 = rb.RoundaboutJunction(vs_main_4, vs_main_1, 0.7, vs_secondary_3, None, queue_junction = True)
vs_junctions = [vs_jnc_1, vs_jnc_2, vs_jnc_3, vs_jnc_4]

In [6]:
# Create the roundabout
roundabout = rb.Roundabout([vs_main_1, vs_main_2, vs_main_3, vs_main_4], 
                           [vs_fw, vs_secondary_1, vs_secondary_2, vs_secondary_3],
                           [vs_bw, None, None, None],vs_junctions)

In [7]:
# Create network:
roads = [vs_main_1, vs_main_2, vs_main_3, vs_main_4, vs_fw, vs_bw]
network = nw.RoadNetwork(roads, [], 50, [roundabout])

In [8]:
densities, queues, _, _ = network.solve_cons_law()

In [9]:
glgif.draw_timed_with_shift(network, densities, interval_seconds = 0.05, output_name = 'roundabout.gif')

: 

In [ ]:
ASFGHKJ

In [ ]:
vs_secondary_1.queue_length

tensor(0., grad_fn=<MaximumBackward0>)

In [ ]:
# vs_jnc_2:
jnc = vs_jnc_2
main_in = jnc.mainline_in
main_out = jnc.mainline_out
second_in = jnc.secondary_in
beta = rb.priority_fnc(main_in.rho[-1])

In [ ]:
densities

{0: {0: tensor([0., 0., 0., 0., 0., 0.]),
  tensor(0.6000, grad_fn=<AddBackward0>): tensor([0.0475, 0.0475, 0.0000, 0.0000, 0.0000, 0.0000], grad_fn=<CopySlices>),
  tensor(1.2000, grad_fn=<AddBackward0>): tensor([0.0495, 0.0495, 0.0234, 0.0224, 0.0221, 0.0221], grad_fn=<CopySlices>),
  tensor(1.8000, grad_fn=<AddBackward0>): tensor([0.0564, 0.0564, 0.0365, 0.0346, 0.0340, 0.0340], grad_fn=<CopySlices>),
  tensor(2.4000, grad_fn=<AddBackward0>): tensor([0.0605, 0.0605, 0.0465, 0.0445, 0.0437, 0.0437], grad_fn=<CopySlices>),
  tensor(3.0000, grad_fn=<AddBackward0>): tensor([0.0653, 0.0653, 0.0535, 0.0519, 0.0511, 0.0511], grad_fn=<CopySlices>),
  tensor(3., grad_fn=<AddBackward0>): tensor([0.0653, 0.0653, 0.0535, 0.0519, 0.0511, 0.0511], grad_fn=<CopySlices>)},
 1: {0: tensor([0., 0., 0., 0., 0., 0.]),
  tensor(0.6000, grad_fn=<AddBackward0>): tensor([0., 0., 0., 0., 0., 0.], grad_fn=<CopySlices>),
  tensor(1.2000, grad_fn=<AddBackward0>): tensor([0., 0., 0., 0., 0., 0.], grad_fn=<CopyS

In [ ]:
t = 50

In [ ]:
main_rho_in = main_in.rho[-1]
main_rho_out = main_out.rho[-1]
main_max_dens = main_in.max_dens
main_gamma = main_in.gamma[main_in.idx]

demands = torch.zeros(2)
demands[0] = main_max_dens * fv.D(main_rho_in, main_gamma)
demands[1] = second_in.demand(t)
# Demands look okay...

In [ ]:
supply = main_max_dens * fv.S(main_rho_out, main_gamma)
supply
# Supply also looks good

tensor(0.1389, grad_fn=<MulBackward0>)

In [ ]:
# alpha = 0.7
alpha = 0.7
out_flux = torch.min((1-alpha)*demands[0] + demands[1])
in_fluxes = torch.zeros(2)
max_main_in = torch.max(beta*supply, supply - demands[1])
in_fluxes[0] = 1/(1-alpha) * torch.min((1-alpha)*demands[0], max_main_in)
max_second_in = torch.max((1-beta)*supply, supply - (1-alpha)*demands[0])
in_fluxes[1] = torch.min(demands[1], max_second_in)

In [ ]:
out_flux # This is okay
in_fluxes # This is wrong!

tensor([4.9659e-05, 1.3887e-01], grad_fn=<CopySlices>)

In [ ]:
out_flux
dt = 1.2
print(main_out.rho)
main_out.update_left_boundary(out_flux, dt)
print(main_out.rho)

tensor([0.0776, 0.0776, 0.0412, 0.0412, 0.0412, 0.0412], grad_fn=<CopySlices>)
tensor([0.4332, 0.4332, 0.0412, 0.0412, 0.0412, 0.0412], grad_fn=<CopySlices>)


In [ ]:
ahfdjk
# glgif.draw_timed_with_shift(network, densities, interval_seconds = 0.05, output_name = 'roundabout.gif')

NameError: name 'ahfdjk' is not defined